In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
trainingDatasets = '/kaggle/input/mri-image-based-brain-tumor-classification/MRI Image Dataset for Brain Tumor/Training'
validationDatasets = '/kaggle/input/mri-image-based-brain-tumor-classification/MRI Image Dataset for Brain Tumor/Validation'

In [ ]:
trainDatasetsGenerator = ImageDataGenerator(rescale = 1./255,
                                           rotation_range = 40,
                                           shear_range = 0.2,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip = True,
                                           fill_mode = "nearest")
validationDatasetsGenerator = ImageDataGenerator(rescale = 1./255)

In [ ]:
trainGenerator = trainDatasetsGenerator.flow_from_directory(
        directory = trainingDatasets,
        target_size = (224,224), 
        shuffle = True,
        batch_size = 32,
        class_mode = "categorical"
    )
validationGenerator = trainDatasetsGenerator.flow_from_directory(
        directory = validationDatasets,
        target_size = (224,224), 
        shuffle = True,
        batch_size = 32,
        class_mode = "categorical"
    )

In [ ]:
base_model = MobileNetV2(weights='imagenet',
                         include_top=False,
                         input_shape=(224,224,3))

In [ ]:
#proposed model
model_data = base_model.output
print(f'Model Data Output: {model_data}')
model_data = GlobalAveragePooling2D()(model_data)
model_data = Dense(1024,activation="relu")(model_data)
final_result = Dense(trainGenerator.num_classes,activation="softmax")(model_data)
model = Model(inputs=base_model.inputs,outputs = final_result)

In [ ]:
#freez trainable part
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.025),
             loss = "categorical_crossentropy",
             metrics = ["Accuracy"])

In [ ]:
history = model.fit(trainGenerator,
                   steps_per_epoch=trainGenerator.n//trainGenerator.batch_size,
                   epochs = 5,
                   validation_data= validationGenerator,
                   validation_steps=validationGenerator.n//validationGenerator.batch_size)

In [ ]:
model_evaluate = model.evaluate(validationGenerator,
                               steps=validationGenerator.n//validationGenerator.batch_size)

In [ ]:
model.save('/kaggle/working/mobilenetv2.h5')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['Accuracy'])
plt.plot(history.history["val_Accuracy"])
plt.title("Model Accuracy (lr=0.025)")
plt.xlabel("Epoch")
plt.legend(['Train','Validation'],loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history["val_loss"])
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.legend(['Train','Validation'],loc = 'upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model_load = load_model("/kaggle/working/mobilenetv2.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
image_data = '/kaggle/input/mri-image-based-brain-tumor-classification/MRI Image Dataset for Brain Tumor/Testing/notumor/Te-noTr_0003.jpg'
images = image.load_img(image_data,target_size = (224,224))
image_to_arrays = image.img_to_array(images)
image_to_arrays = np.expand_dims(image_to_arrays,axis = 0)
image_to_arrays /= 255.

In [ ]:
predictions = model.predict(image_to_arrays)
print(f'Prediction : {predictions[0]}' )

In [ ]:
label_info = ["Glioma","Meningioma","No-tumor","Pituitart"]
predicted_class = label_info[np.argmax(predictions)]
print(f"Predicted Class is: {predicted_class}")

#### Fine tuning

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.0001),
             loss = "categorical_crossentropy",
             metrics = ["Accuracy"])

In [ ]:
history = model.fit(trainGenerator,
                   steps_per_epoch=trainGenerator.n//trainGenerator.batch_size,
                   epochs = 10,
                   validation_data= validationGenerator,
                   validation_steps=validationGenerator.n//validationGenerator.batch_size)

In [ ]:
test_loss, test_accuracy = model.evaluate(validationGenerator,
                               steps = validationGenerator.n//validationGenerator.batch_size)
print(f"Test Loss :{test_loss}\nTest Accuracy :{test_accuracy}")
model_predcitions = model.predict(validationGenerator,
                               steps = validationGenerator.n/validationGenerator.batch_size)
y_actual = validationGenerator.classes
y_pred = np.argmax(model_predcitions, axis = 1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['Accuracy'])
plt.plot(history.history["val_Accuracy"])
plt.title("Model Accuracy (lr=0.0.001)")
plt.xlabel("Epoch")
plt.legend(['Train','Validation'],loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history["val_loss"])
plt.title("Model Accuracy")
plt.xlabel("Epoch")
plt.legend(['Train','Validation'],loc = 'upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_actual, y_pred)
plt.figure(figsize = (10, 5))
sns.heatmap(cm, annot = True, 
           fmt = "d", linewidths = 0.4,
           square = True,
           cmap = "Blues",
           xticklabels = label_info, 
           yticklabels = label_info)
plt.xlabel("Actual Labels")
plt.ylabel("Predcited labels")
plt.title("Brain Image Classifiction With Transfer Learning Confusion Matrix")
plt.show()